<a href="https://colab.research.google.com/github/Itsqy/bulk-croping-image-generator/blob/main/profile_siswa_rectangle_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install pandas requests
!pip install pandas requests pillow
!pip install gdown pandas pillow

In [ ]:

# Import required libraries
import pandas as pd
import requests
import os
import gdown
from PIL import Image
from zipfile import ZipFile
from PIL import Image, ExifTags

# Path to your uploaded CSV file
csv_file_path = "/content/dataset_profil.csv"  # Replace 'your_file.csv' with the uploaded file name

# Read the CSV file
df = pd.read_csv(csv_file_path)

# Column name containing image URLs (replace 'Image_URL' with the name of your column)
url_column = "Image_URL"


# Create a folder to save the images
output_folder = "/content/downloaded_images"
processed_folder = "/content/processed_images"
os.makedirs(output_folder, exist_ok=True)
os.makedirs(processed_folder, exist_ok=True)


In [ ]:


# Function to fix image orientation based on EXIF data
def correct_orientation(image):
    try:
        # Get EXIF data
        exif = image._getexif()
        if exif is not None:
            # Find the orientation tag
            for tag, value in exif.items():
                if tag == 274:  # Orientation tag
                    if value == 3:
                        image = image.rotate(180, expand=True)
                    elif value == 6:
                        image = image.rotate(270, expand=True)
                    elif value == 8:
                        image = image.rotate(90, expand=True)
    except (AttributeError, KeyError, IndexError):
        # If there's no EXIF data or the orientation can't be read, return the image as-is
        pass
    return image

# Function to crop the center part and resize images
def crop_center_and_resize(image_path, output_path, size=(413, 413)):
    with Image.open(image_path) as img:
        # Correct the orientation of the image based on EXIF data
        img = correct_orientation(img)

        # Get the dimensions of the image
        width, height = img.size

        # Define the width of the portion you want to keep (center crop)
        crop_width = min(width, size[0])  # Ensure the crop width is no larger than the desired size

        # Calculate the cropping coordinates to crop the center
        left = (width - crop_width) // 2  # Left side of the crop (center)
        right = left + crop_width  # Right side of the crop (center)

        # Crop the image (center portion)
        img_cropped = img.crop((left, 0, right, height))

        # Resize the cropped image to the desired size (413x413)
        img_resized = img_cropped.resize(size, Image.Resampling.LANCZOS)

        # Save the processed image
        img_resized.save(output_path)
        print(f"Processed and saved: {output_path}")


In [ ]:
# Function to extract Google Drive file ID from the URL
def extract_drive_id(url):
    # Handle drive.google.com links
    if "drive.google.com" in url:
        if "id=" in url:
            return url.split("id=")[-1]
        elif "/d/" in url:
            return url.split("/d/")[1].split("/")[0]
    return None


# Function to crop and resize images
def crop_and_resize(image_path, output_path, size=(1000, 1000)):
    with Image.open(image_path) as img:

        img = correct_orientation(img)

        # Get the dimensions of the image
        width, height = img.size

        # Determine the crop area to create a square
        crop_size = min(width, height)
        left = (width - crop_size) / 2
        top = (height - crop_size) / 2
        right = (width + crop_size) / 2
        bottom = (height + crop_size) / 3

        # Crop and resize the image
        img_cropped = img.crop((left, 0, right, bottom))
        img_resized = img_cropped.resize(size, Image.Resampling.LANCZOS)  # Updated resizing method

        # Save the processed image
        img_resized.save(output_path)
        print(f"Processed and saved: {output_path}")
# Download and process images
for index, row in df.iterrows():
    url = row[url_column]
    try:
        # Extract Google Drive file ID
        # file_id = extract_drive_id(url)
        # if not file_id:
        #     print(f"Invalid Google Drive link: {url}")
        #     continue

        # Download the image using gdown
        output_file_path = os.path.join(output_folder, f"image_{index + 1}.jpg")
        # gdown.download(f"https://drive.google.com/uc?id={file_id}", output_file_path, quiet=False)

        # Process (crop and resize) the image
        processed_file_path = os.path.join(processed_folder, f"image_{index + 1}.jpg")
        crop_and_resize(output_file_path, processed_file_path)
    except Exception as e:
        print(f"Failed to process {url}: {e}")



Processed and saved: /content/processed_images/image_1.jpg
Processed and saved: /content/processed_images/image_2.jpg
Processed and saved: /content/processed_images/image_3.jpg
Processed and saved: /content/processed_images/image_4.jpg
Processed and saved: /content/processed_images/image_5.jpg
Processed and saved: /content/processed_images/image_6.jpg
Processed and saved: /content/processed_images/image_7.jpg
Processed and saved: /content/processed_images/image_8.jpg
Processed and saved: /content/processed_images/image_9.jpg
Processed and saved: /content/processed_images/image_10.jpg
Processed and saved: /content/processed_images/image_11.jpg
Processed and saved: /content/processed_images/image_12.jpg
Processed and saved: /content/processed_images/image_13.jpg
Processed and saved: /content/processed_images/image_14.jpg
Processed and saved: /content/processed_images/image_15.jpg
Processed and saved: /content/processed_images/image_16.jpg
Processed and saved: /content/processed_images/im

In [ ]:
import shutil

# Specify the folder to zip
folder_to_zip = "/content/processed_images"  # Replace with your folder path
output_zip_file = "/content/1080mb_poto_siswa.zip"

# Zip the folder
shutil.make_archive(output_zip_file.replace(".zip", ""), 'zip', folder_to_zip)

print(f"Folder zipped into: {output_zip_file}")


Folder zipped into: /content/1080mb_poto_siswa.zip
